IMPORT THE INPUT FILE

In [ ]:
# Import the Google Colab drive lib
from google.colab import drive
drive.mount('/content/drive')

# Read the input file
input_path = open('/content/drive/My Drive/UFRN/10 - Cursos online/Finding Hidden Messages in DNA (Bioinformatics I)/1 - Where in the Genome Does Replication Begin/E_coli.txt', 'r')
input = input_path.read()

# Print the input
print(input)

THE CODE LOGIC BEGINS HERE!

In [2]:
# ======================================
# Prefix function
# ======================================
def Prefix(input_):
  return input_[0:(len(input_) - 1)].upper()

# ======================================
# LastSymbol function
# ======================================
def LastSymbol(input_):
  return input_[-1].upper()

# ======================================
# SymbolToNumber function
# ======================================
def SymbolToNumber(input_):
  input_ = input_.upper()
  result = ""

  # Iterate over the input string
  for idx, chr in enumerate(input_):
    # Check the lexicographic order
    switcher = {
      "A": "0",
      "C": "1",
      "G": "2",
      "T": "3"
    }
    current_result = switcher.get(input_[idx], "Invalid base")

    # Verify if the current base is valid!
    if (current_result == "Invalid base"):
      return -1
    else:
      result += current_result

  # Return the final result
  return int(result)

# ======================================
# NumberToSymbol function
# ======================================
def NumberToSymbol(input_):
  input_ = str(input_)
  result = ""

  # Iterate over the input number
  for idx, chr in enumerate(input_):
    # Check the lexicographic order
    switcher = {
      "0": "A",
      "1": "C",
      "2": "G",
      "3": "T"
    }
    current_result = switcher.get(input_[idx], "Invalid number")

    # Verify if the current base is valid!
    if (current_result == "Invalid number"):
      return -1
    else:
      result += current_result

  # Return the final result
  return result

# ======================================
# NumberToPattern function
# ======================================
def NumberToPattern(index_, k_):
  # Check if the input k-mer has size 1
  if k_ == 1:
    return NumberToSymbol(index_)

  # Get the Quotient(index, 4)
  prefixIndex = index_ // 4

  # Get the Remainder(index, 4)
  r = index_ % 4

  # Retrieve the equivalent symbol from remainder
  symbol = NumberToSymbol(r)

  # Recursive call with a smaller k-mer
  prefixPattern = NumberToPattern(prefixIndex, k_ - 1)

  # Return concatenation of PrefixPattern with symbol
  return str(prefixPattern) + str(symbol)

# ======================================
# PatternToNumber function
# ======================================
def PatternToNumber(input_):
  # Check if the input Pattern contains at least 1 char
  if len(input_) == 0:
    return 0

  # Reduce the k-mer size until get the prefix and suffix positions
  return 4 * PatternToNumber(Prefix(input_)) + SymbolToNumber(LastSymbol(input_))

# ======================================
# ComputingFrequencies function
# ======================================
def ComputingFrequencies(input_, k_):
  # Declare the frequencyArray
  frequencyArray = []

  # Calculate the loop limits
  loopLimit1 = (4**k_)
  loopLimit2 = len(input_) - (k_-1)

  # Loop 1: Initialize the frequencyArray with zeros
  for idx in range(loopLimit1):
    frequencyArray.append(0)

  # Loop 2: Iterate over the |Text| − k
  for idx in range(loopLimit2):
    current_pattern = input_[idx:(idx + k_)]
    current_number = PatternToNumber(current_pattern)
    frequencyArray[current_number] = frequencyArray[current_number] + 1

  # Return the frequency array
  return frequencyArray

In [14]:
# =======================================================
# ClumpFinding  function
#
# Genome_ = a string containing the genome
# k_ = the k-mer size
# t_ = pattern minimum frequency to be a cluster
# L_ = the slide window
# =======================================================
def ClumpFinding(Genome_, k_, t_, L_):
  # Declare the empty lists
  frequentPatterns = []
  clump = []

  # Calculate the loop limits
  loopLimit1 = (4**k_)
  loopLimit2 = len(Genome_) - L_

  # Get the first slide slideWindow
  slideWindow = Genome_[0:L_]

  # Initialize the frequencyArray
  frequencyArray = ComputingFrequencies(slideWindow, k_)

  # Loop 1: Initialize the clump list
  for idx in range(loopLimit1):
    if frequencyArray[idx] >= t_:
      clump.append(1)
    else:
      clump.append(0)

  # Loop 2: Iterate over the genome
  for idx in range(1, loopLimit2):
    # Update the first pattern from the slideWindow
    first_pattern = Genome_[idx - 1:idx + k_]
    index = PatternToNumber(first_pattern)
    frequencyArray[idx] = frequencyArray[idx] - 1

    # Update the last pattern from the slideWindow
    last_pattern = Genome_[idx + L_ - k_:idx + L_]
    index = PatternToNumber(last_pattern)
    frequencyArray[idx] = frequencyArray[idx] - 1

    # Update the clusters after the slide slideWindow first/last pattern
    if frequencyArray[idx] >= t_:
      clump[idx] = 1

  # Loop 3: Update 
  for idx in range(loopLimit1):
    if clump[idx] == 1:
      # Get the pattern related to the clump
      current_pattern = NumberToPattern(idx, k_)

      # Add the pattern into frequentPatterns list
      frequentPatterns.append(current_pattern)

  # Return the frequentPatterns list
  return frequentPatterns

In [ ]:
# ======================================
# Test section
# ClumpFinding(Genome_, k_, t_, L_):
# ======================================
# Test 1
output1 = ClumpFinding("CGGACTCGACAGATGTGAAGAACGACAATGTGAAGACTCGACACGACAGAGTGAAGAGAAGAGGAAACATTGTAA", 5, 4, 50)
print("(CGGACTCGACAGATGTGAAGAACGACAATGTGAAGACTCGACACGACAGAGTGAAGAGAAGAGGAAACATTGTAA, 5, 4, 50) = " + str(output1))

# Test 2
output2 = ClumpFinding("GTGTTCCTGCCTATTATATGAGACACAGCTATATGACCTCACCAGTCCGGGAGAGCGAGAGCGAGAGCCTAAGCCTAACGCGTGTCGAACATTATTCTCGGGCCTGACGCGATGTACAGGTTAAAATATTCGAAGAATTCCCGGTCTGGGGCGTCAGATGTGAAGGAATAGGTTAGAGCGAGGAATTCCTACTCTCAACAGGCCGCCTCCTAATGAGAAACGGAGTTTACCGAAGAGGATTCCATTCCAGCTTCCTTCCAGCTTCCGACAAAAGTTTAGGCTGGAACTGACCACTGTATAATCCTCTCCAACGCCCGGCGTAGGCCCACCTAACGACGCGCCGTGAGCACCGTTTTTATAGCACCTCTCTGGTCTGTCAGTATGAACCCCTCCTTTATTGGCTCAGGCTGGGGGCGCACGTACGAAGTAGCGAAGGACTGTATTGTCAAATACTTCCACCGATCCCGCCTGGCGGCTGGAGGTTTATCCATCAAGACGGGTCCACAATTCTGGCGTTTTGACGATGTACAGTGGATATAATGGTTTTGGTTGGATATTGATTTGGATATTGTGCGTTACCGTGTTAACTAGGCTGCTCCCAACCCTCGTGAAAATATGCGAGTCGATTACCCGCTTTAGGCCCTCCGACTGGAATTTGTGTTGGAACCGACTACTGACTGGAGCACGTGAAGACCTCCTGTAACGGCTGCTGTTAGTACCGTCCCTCCTAATGCGACCTACCCCTACCCGACGGACCTACCCCCAGCCCCGCCATACCCGGATAACAATCGTCACGGTAATTCAGCTTATCTTGTGCCGGCGCTGGTAGGGGTCTCCTATTGCGCAGAATTGACACAGCCGACTGAGCAAGAAGGCAGGCTAGCGTCGGTCGTTACAGACCACGGGTAGTACGACTAGAACACTACATCGAAGAATTCGTACCATACCGCGTGTAAAACGGCTACGACCTTAAGCCTGTTGTACTGAACTGCGTATAGAGAACCCTCCATAAAGACCCTCTGCCGGCGGTAAACTTGTGAGGGTACTAAATATCGAGCCCAAGTCAATGAGACTGTACGATGAAGCTATGGTTTGTTCCGGCGCCTTGACTTCTACTTGCATTCCTATTCGCATCGCAAACACAGCAAACAGATACCTAGACCCTAAAGAATCATTGGTTGCTTGAACATCTTACCTACTCAGTCTATTTCTGCGGTGGCCTTAAATCGGACGACCATGTCGCCCGCGCCGTTGTCGCGCCGTTGTAACTGTGAAACTCGAATGCCCACGCTAGGGAGGTTTCCCCAATCAACAGCTCCCGTAATCGACGTAATTGGCTATTCCAGTTGCGCGAGTGGTGAAATTAGCGCTTACCTACGCTTGTGAAACTGCATTTATCCCTTCTTTATCCCTCTTGCGATCCGGTCCTAAAAAAAGGTTAGTTCGGGTTAAGCTCTTCCTGCATCGTAAACCTGTACGGCCGAATTCTGTAATCTAATGTCTAGCACAATGAGTGAGTATACCTGGTATAAAGTTCTCCGAAAGTAATACCTTATTCTGTTGTCCCCTGGGATGAGGGATCATGACAATAACGAGTCCATCTCCATCCTGCTCCATCCTGGCCAACTAGAGGCGGCTTGGGTAGGACTGAGTACGCCGGCTCAGTCTTGCTCGATCGAGTTGGATTAGACCCCATATCAGAGTGCAGCACTAGTAGAACATGCCGGATGTTATCGATATGATACGCGCCGCAGCACGGCACGGCAGCACGTTTTCACATATCGTTGCGCGCCGACAAAAGGTAAAGCACTAATATGAACGCATTTCATACAGCAAATGATTCATCCATTAGATATCCCCGGTTGACTACCTAACTATACGCAGGATACTAAGGCCGAATTATGGGGCTGACGTTTTGGGTCTATAGCAAGGACCAATCACCGTTTGTTGTGATTGATTGGCTAGCATAGCATGCTAGCATACTAGCATAGACAG", 
                       9, 4, 26)
print("(Test2, 9, 4, 26) = " + str(output2))

# Test 3 (E_coli genome)
print("E_coli genome size: " + str(len(input)) + " bp")
output3 = ClumpFinding(input, 9, 3, 500)
print("(E_coli genome, 9, 3, 500) = " + str(output3))